In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta, datetime
from DAX.HelpFunctions.get_dax_data import get_dax_data
import seaborn as sns

In [ ]:
hist = get_dax_data()

In [ ]:
hist.tail()

In [ ]:
#quantile levels
tau = [.025, .25, .5, .75, .975]

In [ ]:
#define prediction array
#cols are quantile levels, rows are horizons
pred_baseline = np.zeros((5,5))

In [ ]:
last_t = 1000

for i in range(5):
    ret_str = "ret"+str(i+1)
    
    pred_baseline[i,:] = np.quantile(hist[ret_str].iloc[-last_t:], q=tau)

Visualize the results

In [ ]:
x = np.arange(5)+1
_ = plt.plot(x,pred_baseline, ls="", marker="o", c="black")
_ = plt.xticks(x, x)
_ = plt.plot((x,x),(pred_baseline[:,0], pred_baseline[:,-1]),c='black')

Create Submission DataFrame

In [ ]:
# Returns the five next working days as string
def get_forecast_dates(most_recent_date):
    working_days = []
    current_date = most_recent_date

    while len(working_days) < 5:
        # Increment the current date by one day
        current_date += timedelta(days=1)

        # Check if the current date is a working day (Monday to Friday)
        if current_date.weekday() < 5:
            working_days.append(current_date.date().strftime('%Y-%m-%d'))

    return working_days
        

In [ ]:
df_sub = pd.DataFrame({
    "forecast_date": get_forecast_dates(max(hist.index)),
    "target": "DAX",
    "horizon": [str(i) + " day" for i in (1,2,5,6,7)],
    "q0.025": pred_baseline[:,0],
    "q0.25": pred_baseline[:,1],
    "q0.5": pred_baseline[:,2],
    "q0.75": pred_baseline[:,3],
    "q0.975": pred_baseline[:,4]})
df_sub

In [ ]:
from DAX.Models.baseline import baseline

baseline(hist)